In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [3]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil_price = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')

In [4]:
#preprocessing holidays data
holidays.head()
#filtering out transferred holidays
holidays = holidays[holidays['transferred'] == False]
holidays = holidays.replace(to_replace=['Transfer', 'Bridge', 'Additional'], value='Holiday')
holidays['Holiday_Gen'] = holidays['type']=='Holiday' #holiday or not
holidays['Event_Gen'] = holidays['type']=='Event' #event or not
holidays['Work_Day_Gen'] = holidays['type']=='Work Day'
holidays.drop(columns=["type", "transferred"], inplace=True)

In [5]:
#preprocessing oil price data
oil_price.loc[0, "dcoilwtico"] = oil_price.loc[1, "dcoilwtico"] #fill in the only nan
oil_price.ffill(inplace=True)
oil_price.bfill(inplace=True)

In [6]:
#data preprocessing

scalers = {'scaler_onpromotion' : MinMaxScaler(), 'scaler_day' : MinMaxScaler(), 'saler_month' : MinMaxScaler(),
           'scaler_year' : MinMaxScaler(), 'scaler_dcoilwtico' : MinMaxScaler(), 'scaler_DoW' : MinMaxScaler()}

def preprocess_data(data, stores, transactions, holidays, oil_price, fit_scaler, scalers):
    #data will be empty before adding all dfs
    #merge stores into data to substitute store ids with location data
    data = data.merge(stores, how = 'left', left_on = 'date', right_on = 'date') #left_on = left df (data), right_on = rigt df(stores)
    data['Holiday'] = ((data['Holiday_Gen'] == True)
                      & ((data['city'] == data['locale_name'])
                        | (data['state'] == data['locale_name'])
                        | ('Ecuador' == data['locale_name']))).astype(dtype='float64')
    data['Event'] = ((data['Event_Gen'] == True)
                       & ((data['city'] == data['locale_name'])
                          | (data['state'] == data['locale_name'])
                          | ('Ecuador' == data['locale_name']))).astype(dtype='float64')
    data['Work_Day'] = ((data['Work_Day_Gen'] == True)
                        & ((data['city'] == data['locale_name'])
                           | (data['state'] == data['locale_name'])
                           | ('Ecuador' == data['locale_name']))).astype(dtype='float64')

    #add oil prices to data + fill missing values
    data = data.merge(oil_price, how = 'left', left_on = 'date', right_on = 'date')
    data.sort_values('id', inplace = True) #sort rows by id, modify original df with no copy made
    data['dcoilwtico'] = data['dcoilwtico'].ffill() #forward fill

    #add transactions to data
    data = data.merge(transactions, how = 'left', left_on = ['date', 'store_nbr'], right_on = ['date', 'store_nbr']) #join data and transactions  where date and store_nbr match

    #modify date, pay date info
    data['date'] = pd.to_datetime(data['date']) #turn into math friendly datetime data
    data['day'] = data['date'].dt.day #extract the day from the date and put in a column
    data['month'] = data['date'].dt.month #extract month and put in a column
    data['year'] = data['date'].dt.year #extract year and put in a column
    data['day_of_week'] = data['date'].dt.day_name() #get day of the week NAME (strings)
    data['DoW'] = data['date'].dt.dayofweek #get day of the week as number (monday = 0, int)
    #pay days are on the first and 16th
    data['Pay_Day'] = ((data['date'].dt.day == 1) | (data['date'].dt.day == 16)).astype(float64) #check if the day is 1 or 16 (if its payday)
    #there was an earthquake on 2016-04-16
    data['Earthquake'] = (data['date'] == '2016-04-16').astype(float64)

    #fill nans with proper values
    data.fillna({
        'Holiday_Gen': False, #fill all holiday gen nans with false etc
        'Event_Gen': False,
        'Work_Day_Gen': False,
        'Holiday': 0.0,
        'Event': 0.0,
        'Work_Day': 0.0,
        'Pay_Day': 0.0,
        'Earthquake': 0.0,
        'transactions': 0.0
    }, inplace=True) #change original df directly, no copy is made

    #onehotencoding categorical features: 
    encoder = OneHotEncoder()
    #One_Hot_Encoding = pd.DataFrame(encoder.fit_transform)